In [1]:
using Plots, Measures

include("../src/LoadData.jl")
include("../src/MarginalPosterior.jl")
include("../src/Support.jl")

# Load data
Yin = loadData("NZCOVID_1APR2024");

# Use only the first 100 days when fitting the model
Y = Yin[1:100,:];

In [36]:
function dayofweekModel(θ, Y::DataFrame, opts::Dict)

    # Check that θ are valid
    if sum(θ[3:8]) > 7
        error("Day of week multipliers must sum to less than or equal to 7")
    end

    # Extract frequently used options
    T = opts["T"]
    N = opts["N"]
    L = opts["L"]
    ω = opts["ω"]
    delayDist = opts["delayDist"]
    h = opts["forecastingHorizon"]

    # Extract frequently used parameters
    day_of_week_mult = vcat(θ[3:8], 7 - sum(θ[3:8]))

    # Initialise output matrices
    R = zeros(N, T+h)
    I = zeros(N, T+h)
    W = zeros(N, T+h)

    # And predictive matrices
    μ = zeros(N, T+h)
    C = zeros(N, T+h)

    # Sample from initial distributions
    R[:,1] = rand(opts["pR0"], N)
    I[:,1] = rand(opts["pI0"], N)

    # Run the filter
    for tt = 2:T

        # Project according to the state-space model
        R[:,tt] = exp.(rand.(Normal.(log.(R[:,tt-1]), θ[1])))
        Λ = sum(I[:, (tt-1):-1:1] .* ω[1:(tt-1)]', dims=2) ./ sum(ω[1:(tt-1)])
        I[:,tt] = rand.(Poisson.(R[:,tt] .* Λ))

        # Weight according to the observation model
        μt = day_of_week_mult[1 + tt % 7] * sum(I[:,(tt-1):-1:1] .* delayDist[1:(tt-1)]', dims=2) ./ sum(delayDist[1:(tt-1)])
        r = 1/θ[2]
        p = 1 ./ (1  .+ θ[2] * μt)
        W[:,tt] = pdf.(NegativeBinomial.(r, p), Y.Ct[tt])

        # Resample
        inds = wsample(1:N, W[:,tt], N; replace=true)
        R[:, max(tt - L, 1):tt] = R[inds, max(tt - L, 1):tt]
        I[:, max(tt - L, 1):tt] = I[inds, max(tt - L, 1):tt]

        if opts["predictiveValues"]
            μ[:,tt] = μt
            μ[:,max(tt-L,1):tt] = μ[inds,max(tt-L,1):tt]
        end

    end

    # Fetch predictive values
    if opts["predictiveValues"]
        for tt = 2:T
            r = 1/θ[2]
            p = 1 ./ (1 .+ θ[2] * μ[:,tt])
            C[:,tt] = rand.(NegativeBinomial.(r, p))
        end
    end

    # Run forecast
    if h > 0
        for tt = (T+1):(T+h)
            R[:,tt] = exp.(rand.(Normal.(log.(R[:,tt-1]), θ[1])))
            Λ = sum(I[:, (tt-1):-1:1] .* ω[1:(tt-1)]', dims=2) ./ sum(ω[1:(tt-1)])
            I[:,tt] = rand.(Poisson.(R[:,tt] .* Λ))
            μt = day_of_week_mult[1 + tt % 7] * sum(I[:,(tt-1):-1:1] .* delayDist[1:(tt-1)]', dims=2) ./ sum(delayDist[1:(tt-1)])
            r = 1/θ[2]
            p = 1 ./ (1  .+ θ[2] * μt)
            C[:,tt] = rand.(NegativeBinomial.(r, p))
        end
    end


    # Store output as three-dimensional array
    X = zeros(N, T+h, 3)
    X[:,:,1] = R
    X[:,:,2] = I
    X[:,:,3] = C

    return(X, W)

end

dayofweekModel (generic function with 1 method)

In [24]:
opts = Dict(

    # Bootstrap filter options
    "T" => size(Y, 1), # Number of time-steps
    "N" => 1000, # Number of particles
    "L" => 50, # Fixed-lag resampling length
    "ω" => pdf.(Gamma(2.36, 2.74), 1:128), # Serial interval
    "delayDist" => pdf.(Gamma(5.72, 0.96), 1:200), # Observation delay distribution
    "pR0" => Uniform(0, 10), # Prior on Rt at t = 0
    "pI0" => DiscreteUniform(200, 600), # Prior on I at t = 0
    "predictiveValues" => false, # Whether to calculate predictive cases
    "forecastingHorizon" => 0, # Number of days to forecast

    # PMMH options
    "nChains" => 3, # Number of chains
    "chunkSize" => 100, # Number of iterations
    "maxChunks" => 50, # Maximum number of chunks
    "maxRhat" => 1.05,  # Stopping criterion: maximum Rhat value
    "minESS" => 100, # Stopping criterion: minimum effective sample size
    "showChunkProgress" => true, # Whether to show progress of each chunk
    "propStdDevInit" => sqrt.([0.0002, 7e-6, 0.0005, 0.001, 0.001, 0.001, 0.001, 0.0005]), # Initial proposal standard deviation (this is adaptively fit)
    "paramPriors" => vcat([Uniform(0, 1), Uniform(0, 0.1)], repeat([Uniform(0.1, 2)], 6)),
    "initialParamSamplers" => [Uniform(0.05, 0.07), Uniform(0.01, 0.014), Uniform(0.75, 0.80), Uniform(1.3, 1.35), Uniform(1.2, 1.25), Uniform(1.1, 1.2), Uniform(0.95, 1.00), Uniform(0.9, 0.95)],
    "paramLimits" => vcat([(0, 1), (0, 1)], repeat([(0.1, 2)], 6)),
    "paramNames" => ["sigma", "phi", "Day1", "Day2", "Day3", "Day4", "Day5", "Day6"],

    # Marginal posterior options
    "posteriorNumberOfParticles" => 10000,
    "posteriorParamSamples" => 100

);

In [34]:
# # Run adaptive PMMH to get first estimate of covariance matrix
# include("../src/PMMH.jl")
# (θ, diag) = PMMH(dayofweekModel, Y, opts; verbose=true)
# chains = Chains(θ, opts["paramNames"])
# CSV.write("temp/dayOfWeekParamCovEst.csv", DataFrame(cov(resizeParams(θ))))

# Run manual PMMH using the covariance matrix estimated above
include("../src/PMMHManual.jl")
covEstIn = CSV.read("temp/dayOfWeekParamCovEst.csv", DataFrame)
opts["proposalCovariance"] = Matrix(covEstIn)
opts["windin"] = 501
(θ, diag) = PMMH(dayofweekModel, Y, opts; verbose=true)
chains = Chains(θ, opts["paramNames"])

Running chunk 1 100%|██████████████████████████████████████████████████| Time: 0:00:07
Running chunk 2 100%|██████████████████████████████████████████████████| Time: 0:00:07
Running chunk 3 100%|██████████████████████████████████████████████████| Time: 0:00:07
Running chunk 4 100%|██████████████████████████████████████████████████| Time: 0:00:07
Running chunk 5 100%|██████████████████████████████████████████████████| Time: 0:00:07
Running chunk 6 100%|██████████████████████████████████████████████████| Time: 0:00:07


8×6 DataFrame
 Row │ chunk  pAccept   Rhat     ESS       mean        std        
     │ Int64  Float64   Float64  Float64   Float64     Float64    
─────┼────────────────────────────────────────────────────────────
   1 │     6  0.164983  1.35275   8.30481  0.092895    0.0304272
   2 │     6  0.164983  1.13053  17.9258   0.00970835  0.00155673
   3 │     6  0.164983  1.55616   5.31432  0.776472    0.0238089
   4 │     6  0.164983  1.19013  11.9201   1.32851     0.0364895
   5 │     6  0.164983  1.53124   5.45519  1.21876     0.040908
   6 │     6  0.164983  1.6453    5.03501  1.14429     0.030285
   7 │     6  0.164983  1.40221   6.71497  0.980019    0.0364403
   8 │     6  0.164983  1.61011   5.29544  0.911183    0.0314774


Running chunk 7 100%|██████████████████████████████████████████████████| Time: 0:00:07


8×6 DataFrame
 Row │ chunk  pAccept   Rhat     ESS       mean       std        
     │ Int64  Float64   Float64  Float64   Float64    Float64    
─────┼───────────────────────────────────────────────────────────
   1 │     7  0.191919  1.19938  13.8964   0.0891693  0.0276468
   2 │     7  0.191919  1.17755  15.746    0.0104399  0.00189814
   3 │     7  0.191919  1.1194   20.0347   0.779227   0.0225196
   4 │     7  0.191919  1.60564   5.23239  1.34916    0.0500641
   5 │     7  0.191919  1.28114   8.6333   1.20266    0.0434494
   6 │     7  0.191919  1.23733  10.2551   1.14575    0.0273673
   7 │     7  0.191919  1.14736  17.4746   0.980666   0.0303515
   8 │     7  0.191919  1.24321  10.928    0.905658   0.0324928


Running chunk 8 100%|██████████████████████████████████████████████████| Time: 0:00:07

...

Running chunk 27 100%|██████████████████████████████████████████████████| Time: 0:00:07


8×6 DataFrame
 Row │ chunk  pAccept   Rhat     ESS      mean       std        
     │ Int64  Float64   Float64  Float64  Float64    Float64    
─────┼──────────────────────────────────────────────────────────
   1 │    27  0.191919  1.01398  119.23   0.0742074  0.02295
   2 │    27  0.191919  1.02054  176.054  0.0105169  0.00204856
   3 │    27  0.191919  1.04344  114.288  0.776477   0.024182
   4 │    27  0.191919  1.02094   94.754  1.33632    0.0360198
   5 │    27  0.191919  1.02255  138.613  1.21289    0.0315946
   6 │    27  0.191919  1.00814  111.565  1.15098    0.0297815
   7 │    27  0.191919  1.01914  163.537  0.98504    0.026475
   8 │    27  0.191919  1.01245  124.925  0.90657    0.0279787


Running chunk 28 100%|██████████████████████████████████████████████████| Time: 0:00:07


8×6 DataFrame
 Row │ chunk  pAccept   Rhat     ESS      mean       std        
     │ Int64  Float64   Float64  Float64  Float64    Float64    
─────┼──────────────────────────────────────────────────────────
   1 │    28  0.107744  1.01389  134.299  0.073893   0.0229657
   2 │    28  0.107744  1.02442  176.958  0.0104749  0.00207633
   3 │    28  0.107744  1.03626  122.229  0.776237   0.0242086
   4 │    28  0.107744  1.01872  104.76   1.33688    0.0357231
   5 │    28  0.107744  1.01326  150.093  1.21227    0.0318306
   6 │    28  0.107744  1.01779  109.32   1.15142    0.0303468
   7 │    28  0.107744  1.01351  172.813  0.984716   0.026595
   8 │    28  0.107744  1.01081  140.351  0.906893   0.0278606


Chains MCMC chain (2300×8×3 Array{Float64, 3}):

Iterations        = 1:1:2300
Number of chains  = 3
Samples per chain = 2300
parameters        = sigma, phi, Day1, Day2, Day3, Day4, Day5, Day6

Summary Statistics
  parameters      mean       std   naive_se      mcse        ess      rhat 
      Symbol   Float64   Float64    Float64   Float64    Float64   Float64 

       sigma    0.0739    0.0230     0.0003    0.0017   134.2991    1.0139
         phi    0.0105    0.0021     0.0000    0.0001   176.9583    1.0244
        Day1    0.7762    0.0242     0.0003    0.0018   122.2287    1.0363
        Day2    1.3369    0.0357     0.0004    0.0029   104.7601    1.0187
        Day3    1.2123    0.0318     0.0004    0.0024   150.0928    1.0133
        Day4    1.1514    0.0303     0.0004    0.0023   109.3205    1.0178
        Day5    0.9847    0.0266     0.0003    0.0018   172.8135    1.0135
        Day6    0.9069    0.0279     0.0003    0.0020   140.3510    1.0108

Quantiles
  parameters      2.5%  

In [37]:
# Fetch marginal posterior
opts["predictiveValues"] = true
opts["forecastingHorizon"] = 28
X = marginalPosterior(dayofweekModel, θ, Y, opts);

In [38]:
# Save model results
using HDF5
if isfile("temp/dayofweekResults.h5")
    rm("temp/dayofweekResults.h5")
end
h5write("temp/dayofweekResults.h5", "X", X)
h5write("temp/dayofweekResults.h5", "theta", θ)